In [5]:
import random
import time
class Player:
    def __init__(self, name, num_of_cards):
        """
        The base player class of the game
        Inputs
        -----------
        name = (str) player's name
        num_of_cards = (int) number of cards in the deck
        """
        self.name = name
        self.deck_count = num_of_cards
        self.target = self.deck_count * 2 - 1
        self.cards = []
        self.erases_remaining = self.deck_count // 5
        self.has_stopped = False
            

    def draw_card(self, card):
        """
        draws a card, and adds it to player cards
        Input
        -------------
        card: (int) the card to be added
        """
        self.cards.append(card)

    def print_info(self):
        """
        prints info of the player
        """
        print(f"{self.name}'s cards: ", end='')
        for c in self.cards:
            print(f'{c}, ', end='')
        print(f'sum: {sum(self.cards)}')
    
    def get_margin(self):
        """
        returns the margin left to target by the player
        Output
        ----------
        (int) margin to target
        """
        return self.target - sum(self.cards)
    
    def cpu_play(self, seen_cards, deck, enemies_cards):
        """
        The function for cpu to play the game
        Inputs
        ----------
        seen_cards:     (list of ints) the cards that have been seen until now
        deck:           (list of ints) the remaining playing deck of the game
        enemies_cards:  (list of ints) the cards that the enemy currently has.
        Output
        ----------
        (str) a command given to the game
        
        """
        if (len(deck) > 0):
            next_card_in_deck = deck[0]
        else:
            next_card_in_deck = 0
        if (len(deck) > 1):
            next_enemy_card_in_deck = deck[1]
        else:
            next_enemy_card_in_deck = 0
        amount_to_target = self.target - sum(self.cards)
        amount_with_next_card = self.target - (sum(self.cards) + next_card_in_deck)
        enemies_amount_to_target = self.target - sum(enemies_cards)
        enemies_amount_with_next_card = self.target - (sum(enemies_cards) + next_enemy_card_in_deck)
        _stop_condition = amount_to_target < next_card_in_deck and self.erases_remaining <= 0
        _draw_condition_1 = next_card_in_deck != 0
        _draw_condition_2 = amount_with_next_card >= 0
        _erase_condition = self.erases_remaining > 0
        _erase_self_condition = amount_to_target < 0
        _erase_opponent_condition_or = enemies_amount_to_target < (self.target // 7)
        _erase_opponent_condition_or_2 = enemies_amount_with_next_card < (self.target // 7) 
        _erase_opponent_condition_or_3 = enemies_amount_with_next_card <= amount_with_next_card
        _erase_opponent_condition_or_4 = enemies_amount_to_target <= amount_to_target
        _erase_opponent_condition = _erase_opponent_condition_or or _erase_opponent_condition_or_2 or _erase_opponent_condition_or_3
        _erase_opponent_condition = _erase_opponent_condition or _erase_opponent_condition_or_4 
        if (_stop_condition):
            return 'stop'
        elif (_draw_condition_1 and _draw_condition_2):
            return 'draw'
        elif(_erase_self_condition and _erase_condition):
            return 'erase_self'
        elif(_erase_opponent_condition and _erase_condition):
            return 'erase_opponent'
        else:
            return 'stop'
    
    def erase(self, target):
        """
        erases the last card of the target
        Input
        ---------
        target: (Player obj) the player whos last card is about to be erased
        """
        if (len(target.cards) == 0):
            print(f'{target.name} has no more eraseble cards!')
            return
        if (self.erases_remaining > 0):
            self.erases_remaining -= 1
            card = target.cards.pop(-1)
            print(f'{self.name} erased {card} from {target.name}\'s deck!')
            return
        print(f'{self.name} has no more erases remaining!')

    def get_player_cards(self):
        return self.cards

    def get_erases_remained(self):
        return self.erases_remaining

class Blacksin:
    def __init__(self, deck_count=21):
        """
        The main game class
        Inputs
        -----------
        deck_count = (int) number of cards in the deck
        """
        self.deck_count = deck_count
        self.target = self.deck_count * 2 - 1
        self.player = Player('player', deck_count)
        self.opponent = Player('opponent', deck_count)
        self.deck = self.shuffle_cards()
        self.seen_cards = []
    
    def shuffle_cards(self):
        """ 
        shuffles cards for deck creation
        """
        return list(random.sample(range(1, self.deck_count + 1), self.deck_count))

    def draw_card(self):
        """ 
        draws a card from deck, if non is remaining, ends the game.
        """
        if (len(self.deck) > 0):
            card = self.deck.pop(0)
            self.seen_cards.append(card)
            return card
        print('The deck is empty! ending game...')
        self.opponent.has_stopped = True
        self.player.has_stopped = True
        return -1

    def handout_cards(self):
        """ 
        hands out cards to players
        """
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
        self.player.draw_card(self.draw_card())
        self.opponent.draw_card(self.draw_card())
    
    def handle_input(self, _input, player):
        """ 
        handles input
        Input
        ------------
        _input: (str) input given by the player
        player: (Player obj)the player that is giving the input
        
        """
        if (player is self.player):
            opponent = self.opponent
        else:
            opponent = self.player
        if (_input == 'stop' or _input == 's'):
            player.has_stopped = True
            print(f'{player.name} has stopped')
        elif (_input == 'draw' or _input == 'd'):
            card = self.draw_card()
            if (card == -1): return True
            player.draw_card(card)
            print(f'{player.name} drawed a card: {card}')
        elif ((_input == 'erase_self' or _input == 'es')):
            player.erase(player)
        elif ((_input == 'erase_opponent' or _input == 'eo')):
            player.erase(opponent)
        else:
            print('ERROR: unknown command')
            return False
        return True

    def move(self,lastMove,turn,orgCards,playerCards,opponentCards,playerStopped,opponentStopped,playerEraseRem,opponentEraseRem):
        if (playerStopped and opponentStopped) or (orgCards == [] and playerEraseRem <= 0 and opponentEraseRem <= 0) \
            or (not turn and opponentStopped and sum(playerCards) - sum(opponentCards) > 0) or (turn and playerStopped and sum(opponentCards) - sum(playerCards) > 0):
            point = sum(playerCards) - sum(opponentCards) > 0
            return [point,lastMove,"s"]

        whatsNewTurn = 1 - turn
        if playerStopped: whatsNewTurn = 1
        elif opponentStopped: whatsNewTurn = 0

        childPoint = None
        if len(orgCards) > 0:
            orgCopyCards = orgCards[:]
            if not turn and not playerStopped and sum(playerCards) + orgCards[0] <= self.target:
                playerCopyCards = playerCards[:]
                playerCopyCards.append(orgCopyCards.pop(0))
                childPoint = self.move("d",whatsNewTurn,orgCopyCards[:],playerCopyCards[:],opponentCards[:],playerStopped,\
                    opponentStopped,playerEraseRem,opponentEraseRem)
                if childPoint[0]: return [childPoint[0],lastMove,"d"]
            elif turn and not opponentStopped and sum(opponentCards) + orgCards[0] <= self.target:
                oppCpyCards = opponentCards[:]
                oppCpyCards.append(orgCopyCards.pop(0))
                childPoint = self.move("d",whatsNewTurn,orgCopyCards[:],playerCards[:],oppCpyCards[:],playerStopped,\
                    opponentStopped,playerEraseRem,opponentEraseRem)
                if not childPoint[0]: return [childPoint[0],lastMove,"d"]

        if not turn and not playerStopped and len(playerCards) > 0 and playerEraseRem > 0:
            playerCopyCards = playerCards[:]
            playerCopyCards.pop(-1)
            childPoint = self.move("es",whatsNewTurn,orgCards[:],playerCopyCards[:],opponentCards[:],playerStopped,opponentStopped,playerEraseRem - 1,opponentEraseRem)
            if childPoint[0]: return [childPoint[0],lastMove,"es"]
        elif turn and not opponentStopped and len(opponentCards) > 0 and opponentEraseRem > 0:
            oppCpyCards = opponentCards[:]
            oppCpyCards.pop(-1)
            childPoint = self.move("es",whatsNewTurn,orgCards[:],playerCards[:],oppCpyCards[:],playerStopped,opponentStopped,playerEraseRem,opponentStopped - 1)
            if not childPoint[0]: return [childPoint[0],lastMove,"es"]
 
        if not turn and not playerStopped and len(opponentCards) > 0 and playerEraseRem > 0:
            oppCpyCards = opponentCards[:]
            oppCpyCards.pop(-1)
            childPoint = self.move("eo",whatsNewTurn,orgCards[:],playerCards[:],oppCpyCards[:],playerStopped,opponentStopped,playerEraseRem - 1,opponentEraseRem)
            if childPoint[0]: return [childPoint[0],lastMove,"eo"]
        elif turn and not opponentStopped and len(playerCards) > 0 and opponentEraseRem > 0:
            playerCopyCards = playerCards[:]
            playerCopyCards.pop(-1)
            childPoint = self.move("eo",whatsNewTurn,orgCards[:],playerCopyCards[:],opponentCards[:],playerStopped,opponentStopped,playerEraseRem,opponentEraseRem - 1)
            if not childPoint[0]: return [childPoint[0],lastMove,"eo"]
 
        if not turn and not playerStopped:
            childPoint = self.move("s",whatsNewTurn,orgCards[:],playerCards[:],opponentCards[:],True,opponentStopped,playerEraseRem,opponentEraseRem)
            if childPoint[0]: return [childPoint[0],lastMove,"s"]
        elif turn and not opponentStopped:
            childPoint = self.move("s",whatsNewTurn,orgCards[:],playerCards[:],opponentCards[:],playerStopped,True,playerEraseRem,opponentEraseRem)
            if not childPoint[0]: return [childPoint[0],lastMove,"s"]

        if childPoint == None:
            point = sum(playerCards) - sum(opponentCards) > 0
            return [point,lastMove,"s"]

        point = childPoint[0]
        bestMove = childPoint[2]
        return [point,lastMove,bestMove]


    def get_player_input(self):
        move = self.move(None,0,self.deck,self.player.cards,self.opponent.cards,self.player.has_stopped, \
            self.opponent.has_stopped,self.player.erases_remaining,self.opponent.erases_remaining)[2]
        self.handle_input(move, self.player)
            
    def opponent_play(self):
        """
        function for opponent to play it's turn
        """
        try:
            opponent_input = self.opponent.cpu_play(self.seen_cards, self.deck, self.player.cards)
        except:
            opponent_input = 'stop'
        self.handle_input(opponent_input, self.opponent)

    def check_for_winners(self):
        """
        checks for winners.
        Output
        -----------
        (int) returns 1 if player wins, 0 if draw and -1 if opponent wins
        """
        self.opponent.print_info()
        self.player.print_info()
        player_margin = self.player.get_margin()
        opponent_margin = self.opponent.get_margin()
        player_win_condition_1 = opponent_margin < 0 and player_margin >= 0
        player_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin < opponent_margin
        draw_condition_1 = opponent_margin < 0 and player_margin < 0
        draw_condition_2 = opponent_margin >= 0 and player_margin >= 0 and player_margin == opponent_margin
        opponent_win_condition_1 = player_margin < 0 and opponent_margin >= 0
        opponent_win_condition_2 = opponent_margin >=0 and player_margin >= 0 and player_margin > opponent_margin
        if (player_win_condition_1 or player_win_condition_2):
            print(f'the winner is the {self.player.name}!')
            return 1
        elif(draw_condition_1 or draw_condition_2):
            print('the game ends in a draw!')
            return 0
        elif(opponent_win_condition_1 or opponent_win_condition_2):
            print(f'the winner is the {self.opponent.name}!')
            return -1
        else:
            print('an error has accurred! exiting...')
            exit()

    def print_deck(self):
        """
        prints the current deck of the game
        """
        print('full deck: [top] ', end='')
        for i in self.deck:
            print(i, end=' ')
        print('[bottom]')

    def run(self):
        """
        main function to run the game with
        """
        print('\nstarting game... shuffling... handing out cards...')
        print(f'remember, you are aiming for nearest to: {self.target}')
        self.print_deck()
        self.handout_cards()
        turn = 0
        while(not self.player.has_stopped or not self.opponent.has_stopped):
            if (turn == 0):
                if (not self.player.has_stopped):                    
                    self.opponent.print_info()
                    self.player.print_info()
                    self.get_player_input()
                    print()
            else:
                if (not self.opponent.has_stopped):
                    print('opponent playing...')
                    self.opponent_play()
                    print()
            turn = 1 - turn
        print('\nand the winner is...')
        return self.check_for_winners()

if __name__ == "__main__":
    t1 = time.time()
    numOfWin = 0
    for i in range(500):
        game = Blacksin(deck_count=21)
        result = game.run()
        if result: numOfWin += 1
    t2 = time.time()
    percentage = numOfWin/(i+1)*100
    winner = "opponent"
    if percentage > 50:
        winner = "player"
    print("{}% in {} games and the ultimate winner is {}!".format(percentage,i+1,winner))
    print("Time is {} sec.".format(t2 - t1))



starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 4 13 1 5 12 16 20 15 3 17 18 7 6 19 10 2 14 8 11 9 21 [bottom]
opponent's cards: 13, 5, sum: 18
player's cards: 4, 1, sum: 5
player drawed a card: 12

opponent playing...
opponent drawed a card: 16

opponent's cards: 13, 5, 16, sum: 34
player's cards: 4, 1, 12, sum: 17
player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 13, 5, 16, sum: 34
player's cards: 4, 1, 12, sum: 17
player drawed a card: 15

opponent playing...
opponent drawed a card: 3

opponent's cards: 13, 5, 16, 3, sum: 37
player's cards: 4, 1, 12, 15, sum: 32
player erased 15 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 13, 5, 16, 3, sum: 37
player's cards: 4, 1, 12, sum: 17
player drawed a card: 17

opponent's cards: 13, 5, 16, 3, sum: 37
player's cards: 4, 1, 12, 17, sum: 34
player erased 17 from player's deck!

opponent's ca

opponent's cards: 1, 2, 3, 16, sum: 22
player's cards: 15, 6, sum: 21
player drawed a card: 9

opponent playing...
opponent drawed a card: 4

opponent's cards: 1, 2, 3, 16, 4, sum: 26
player's cards: 15, 6, 9, sum: 30
player drawed a card: 11

opponent playing...
opponent drawed a card: 14

opponent's cards: 1, 2, 3, 16, 4, 14, sum: 40
player's cards: 15, 6, 9, 11, sum: 41
player erased 11 from player's deck!

opponent playing...
opponent erased 9 from player's deck!

opponent's cards: 1, 2, 3, 16, 4, 14, sum: 40
player's cards: 15, 6, sum: 21
player drawed a card: 18

opponent playing...
opponent erased 18 from player's deck!

opponent's cards: 1, 2, 3, 16, 4, 14, sum: 40
player's cards: 15, 6, sum: 21
player drawed a card: 7

opponent playing...
opponent has stopped

opponent's cards: 1, 2, 3, 16, 4, 14, sum: 40
player's cards: 15, 6, 7, sum: 28
player erased 7 from player's deck!

opponent's cards: 1, 2, 3, 16, 4, 14, sum: 40
player's cards: 15, 6, sum: 21
player drawed a card: 19



player's cards: 19, 20, sum: 39
the winner is the player!

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 8 19 20 21 3 11 16 18 6 17 14 15 13 5 9 1 2 7 12 10 4 [bottom]
opponent's cards: 19, 21, sum: 40
player's cards: 8, 20, sum: 28
player drawed a card: 3

opponent playing...
opponent erased 3 from player's deck!

opponent's cards: 19, 21, sum: 40
player's cards: 8, 20, sum: 28
player drawed a card: 11

opponent playing...
opponent erased 11 from player's deck!

opponent's cards: 19, 21, sum: 40
player's cards: 8, 20, sum: 28
player erased 20 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 19, 21, sum: 40
player's cards: 8, sum: 8
player drawed a card: 16

opponent's cards: 19, 21, sum: 40
player's cards: 8, 16, sum: 24
player erased 16 from player's deck!

opponent's cards: 19, 21, sum: 40
player's cards: 8, sum: 8
player drawed a card: 18

opponent's cards: 19, 21, sum: 40
player's car

player erased 11 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, sum: 19
player drawed a card: 16

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 16, sum: 35
player drawed a card: 1

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 16, 1, sum: 36
player erased 1 from player's deck!

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 16, sum: 35
player erased 16 from player's deck!

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, sum: 19
player drawed a card: 7

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 7, sum: 26
player drawed a card: 3

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 7, 3, sum: 29
player erased 3 from player's deck!

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 7, sum: 26
player drawed a card: 15

opponent's cards: 18, 6, 13, sum: 37
player's cards: 17, 2, 7, 15, sum: 41
player has stopped


and the winner

full deck: [top] 14 8 16 4 19 21 10 17 15 3 11 9 7 20 18 5 12 6 13 2 1 [bottom]
opponent's cards: 8, 4, sum: 12
player's cards: 14, 16, sum: 30
player erased 16 from player's deck!

opponent playing...
opponent drawed a card: 19

opponent's cards: 8, 4, 19, sum: 31
player's cards: 14, sum: 14
player drawed a card: 21

opponent playing...
opponent drawed a card: 10

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, 21, sum: 35
player erased 21 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, sum: 14
player drawed a card: 17

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, 17, sum: 31
player erased 17 from player's deck!

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, sum: 14
player drawed a card: 15

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, 15, sum: 29
player drawed a card: 3

opponent's cards: 8, 4, 19, 10, sum: 41
player's cards: 14, 15, 3, sum: 32
player era

...

player drawed a card: 18

opponent playing...
opponent drawed a card: 12

opponent's cards: 1, 9, 12, sum: 22
player's cards: 3, 15, 18, sum: 36
player erased 18 from player's deck!

opponent playing...
opponent drawed a card: 8

opponent's cards: 1, 9, 12, 8, sum: 30
player's cards: 3, 15, sum: 18
player drawed a card: 20

opponent playing...
opponent erased 20 from player's deck!

opponent's cards: 1, 9, 12, 8, sum: 30
player's cards: 3, 15, sum: 18
player drawed a card: 19

opponent playing...
opponent drawed a card: 5

opponent's cards: 1, 9, 12, 8, 5, sum: 35
player's cards: 3, 15, 19, sum: 37
player erased 19 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 1, 9, 12, 8, 5, sum: 35
player's cards: 3, 15, sum: 18
player drawed a card: 14

opponent's cards: 1, 9, 12, 8, 5, sum: 35
player's cards: 3, 15, 14, sum: 32
player drawed a card: 7

opponent's cards: 1, 9, 12, 8, 5, sum: 35
player's cards: 3, 15, 14, 7, sum: 39
player has stopped


and the winn

full deck: [top] 15 1 20 16 5 13 3 18 17 8 2 6 14 9 10 11 12 21 7 4 19 [bottom]
opponent's cards: 1, 16, sum: 17
player's cards: 15, 20, sum: 35
player drawed a card: 5

opponent playing...
opponent drawed a card: 13

opponent's cards: 1, 16, 13, sum: 30
player's cards: 15, 20, 5, sum: 40
player erased 5 from player's deck!

opponent playing...
opponent drawed a card: 3

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, 20, sum: 35
player erased 20 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, sum: 15
player drawed a card: 18

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, 18, sum: 33
player erased 18 from player's deck!

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, sum: 15
player drawed a card: 17

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, 17, sum: 32
player drawed a card: 8

opponent's cards: 1, 16, 13, 3, sum: 33
player's cards: 15, 17, 8, sum: 40
play


starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 7 13 20 11 9 8 10 2 18 16 4 17 1 6 14 21 19 12 15 3 5 [bottom]
opponent's cards: 13, 11, sum: 24
player's cards: 7, 20, sum: 27
player drawed a card: 9

opponent playing...
opponent drawed a card: 8

opponent's cards: 13, 11, 8, sum: 32
player's cards: 7, 20, 9, sum: 36
player erased 9 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 13, 11, 8, sum: 32
player's cards: 7, 20, sum: 27
player drawed a card: 10

opponent's cards: 13, 11, 8, sum: 32
player's cards: 7, 20, 10, sum: 37
player has stopped


and the winner is...
opponent's cards: 13, 11, 8, sum: 32
player's cards: 7, 20, 10, sum: 37
the winner is the player!

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 3 20 15 14 4 10 12 18 11 7 16 6 19 21 9 17 1 5 8 13 2 [bottom]
opponent's cards: 20, 14, sum: 34
player's cards: 3, 15

player's cards: 3, 10, 4, 20, sum: 37
player erased 20 from player's deck!

opponent's cards: 9, 11, 2, 15, sum: 37
player's cards: 3, 10, 4, sum: 17
player drawed a card: 5

opponent's cards: 9, 11, 2, 15, sum: 37
player's cards: 3, 10, 4, 5, sum: 22
player drawed a card: 16

opponent's cards: 9, 11, 2, 15, sum: 37
player's cards: 3, 10, 4, 5, 16, sum: 38
player has stopped


and the winner is...
opponent's cards: 9, 11, 2, 15, sum: 37
player's cards: 3, 10, 4, 5, 16, sum: 38
the winner is the player!

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 13 3 5 17 15 18 21 1 2 7 20 9 6 8 16 19 14 11 4 12 10 [bottom]
opponent's cards: 3, 17, sum: 20
player's cards: 13, 5, sum: 18
player drawed a card: 15

opponent playing...
opponent drawed a card: 18

opponent's cards: 3, 17, 18, sum: 38
player's cards: 13, 5, 15, sum: 33
player erased 15 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 3, 17, 1

opponent's cards: 18, 2, sum: 20
player's cards: 1, 8, 12, sum: 21
the winner is the player!

starting game... shuffling... handing out cards...
remember, you are aiming for nearest to: 41
full deck: [top] 7 4 19 21 6 9 20 5 10 18 16 3 15 17 2 13 12 1 11 14 8 [bottom]
opponent's cards: 4, 21, sum: 25
player's cards: 7, 19, sum: 26
player drawed a card: 6

opponent playing...
opponent drawed a card: 9

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7, 19, 6, sum: 32
player erased 6 from player's deck!

opponent playing...
opponent has stopped

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7, 19, sum: 26
player erased 19 from player's deck!

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7, sum: 7
player drawed a card: 20

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7, 20, sum: 27
player drawed a card: 5

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7, 20, 5, sum: 32
player erased 5 from player's deck!

opponent's cards: 4, 21, 9, sum: 34
player's cards: 7,

<strong>
    برای گره های ترمینال اگر از حریف جلوتر باشیم به آن امتیاز 1 میدهیم اما اگر عقب یا مساوی باشیم امتیاز منفی 1 میگیریم و برای گره های بالاتر بین بچه ها بسته به نوبت ماکسیمم یا مینیمم میگیریم<br/>
بعد از هرس زمان و حافظه استفاده شده به شدت بهبود پیدا میکند و هرس به این صورت است که خیلی از استیت ها که به هر حال یا به شکست ما منجر میشود یا ممکن است راه دوم بردن برای یک استیت باشد که به درد ما نمیخورد را پیمایش نمیکنیم<br/>
ترتیب ها به این صورت است که اگر توانست کارت بکشد، در مرحله بعد اگر توانست کارت خود را حذف کند، و در وهله بعدی اگر توانست کارت حریف را حذف کند و در مرحله آخر اگر توانست استاپ کند.<br/>
    ترتیب مهم نیست چون چند تا جواب برای مسئله داریم و به دلخواه هست.<br/>
    </strong>
    <strong>
    اگر ما ترتیب کارت های روز زمین یا کارت های داخل دست حریف را نتوانیم ببینیم، میشود از الگوریتم <br/>
    Expectimax <br/>
    استفاده کرد. به این صورت است که فقط می دانیم کدام کارت ها در کارت های روی زمین نیستند ( کارت هایی که دست خودمان است ) و می توانیم میانگین از این کارت های باقی مانده بگیریم و در نظر بگیریم که کارت بعدی این کارت است و با آن به جلو برویم.
    </strong>